## Assigment 5 
Karl-Axel Jönsson BME4

In [2]:
import os
from glob import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from matplotlib import pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #To get rid of annoying tf messages
import tensorflow as tf 
from sklearn.model_selection import KFold
from numpy import argmax


In [3]:
base_dir = '/Users/karl-axeljonsson/Downloads/HAM1000_64_64/'
df=pd.read_csv('/Users/karl-axeljonsson/Downloads/HAM1000_64_64/dataframe.csv')
display(df)
length = df.shape[0]
label_dict = {'nv': 0,'mel': 1,'bkl': 2, 'bcc': 3, 'akiec': 4,'vasc': 5,'df': 6 }
df['labels'] = df['dx'].map(label_dict) #byter ut mot label_dict
X = np.zeros((length,64,64,3))   
y = np.zeros((length))

k = 0
for i in df['image_id']: 
  
  X[k,:,:,:] = np.asarray(Image.open(base_dir +  i + '.jpg'))
  k= k+1

y = np.asarray(df['labels'])  


,Unnamed: 0,lesion_id,image_id,dx,dx_type,age,sex,localization
0,6358,HAM_0005358,ISIC_0026982,nv,follow_up,50.0,male,lower extremity
1,9592,HAM_0003236,ISIC_0033053,nv,consensus,30.0,female,upper extremity
2,6205,HAM_0001075,ISIC_0028743,nv,follow_up,30.0,female,back
3,7639,HAM_0004882,ISIC_0034178,nv,histo,20.0,male,chest
4,9123,HAM_0004553,ISIC_0028673,nv,histo,45.0,female,lower extremity
...,...,...,...,...,...,...,...,...
996,1196,HAM_0005112,ISIC_0024386,df,consensus,40.0,female,lower extremity
997,1132,HAM_0002706,ISIC_0029783,df,histo,40.0,male,lower extremity
998,1134,HAM_0004371,ISIC_0030011,df,histo,45.0,female,upper extremity
999,1185,HAM_0004648,ISIC_0025911,df,consensus,65.0,female,lower extremity


In [5]:
X = X.astype('float32')
X /=255.0
yidx = y
y = tf.keras.utils.to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=10)





(1001, 7, 2)
(1001, 7)


In [142]:

n_folds=5
kfold=KFold(n_folds, shuffle=True, random_state=1)
scores, histories = list(), list()
best_model = []
best_score = 0

for train_ix, test_ix in kfold.split(X_train):
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu', padding='same', kernel_initializer='he_uniform',input_shape=(64,64,3)))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(49,activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(7, activation='softmax'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    

    trainX, trainY, testX, testY = X_train[train_ix], y_train[train_ix], X_train[test_ix],y_train[test_ix]
    history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
    test_results = model.evaluate(testX, testY, verbose=0)
    print("Test accuracy is {:.4f} ".format(test_results[1]))
      # stores scores
    scores.append(test_results[1])
    histories.append(history)
    if test_results[1] > best_score:
     best_score = test_results[1]
     best_model = model

model.summary()


Test accuracy is 0.7437 
Test accuracy is 0.6438 
Test accuracy is 0.6187 
Test accuracy is 0.6812 
Test accuracy is 0.6375 
Model: "sequential_136"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_366 (Conv2D)         (None, 64, 64, 64)        1792      
                                                                 
 max_pooling2d_296 (MaxPooli  (None, 32, 32, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_367 (Conv2D)         (None, 32, 32, 32)        18464     
                                                                 
 conv2d_368 (Conv2D)         (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_297 (MaxPooli  (None, 16, 16, 32)       0         
 ng2D)                                                     

In [148]:
for i in range(7):
    img=(X[np.where(yidx == i)[0][0], :, :])
    img=img.reshape(1, 64, 64, 3)
    y_hat = model.predict(img)
    print("The model predicts {:d} ".format(argmax(y_hat)))
    
    


1/1 [==============================] - 0s 41ms/step
The model predicts 0 
1/1 [==============================] - 0s 34ms/step
The model predicts 0 
1/1 [==============================] - 0s 34ms/step
The model predicts 0 
1/1 [==============================] - 0s 33ms/step
The model predicts 0 
1/1 [==============================] - 0s 37ms/step
The model predicts 0 
1/1 [==============================] - 0s 45ms/step
The model predicts 0 
1/1 [==============================] - 0s 32ms/step
The model predicts 0 


It is not to my surprise that the model only predicts 0 (nv) because of 671/1000 of the data has the dx type nv whilst for an example, 10/1000 have the dx type df. The other dx types is also way too few and to get a better model more data is needed (at least data that have variance) and more preprocessing. 